In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh as bok
import nltk
import json
import csv
import pylrc
from sklearn import *
from io import StringIO

In [2]:
metadata = pd.read_csv('K:\Multi-modal-MER\PMEmo\PMEmo2019\metadata.csv')

In [3]:
metadata

,musicId,fileName,title,artist,album,duration,chorus_start_time,chorus_end_time
0,1,1.mp3,Good Drank,2 Chainz,"Def Jam Presents: Direct Deposit, Vol. 2",32.10,02:35,03:05
1,4,4.mp3,X Bitch (feat. Future),21 Savage,Savage Mode,28.09,03:00,03:26
2,5,5.mp3,No Heart,21 Savage,Savage Mode,84.23,00:41,02:03
3,6,6.mp3,Red Opps,21 Savage,Red Opps,29.53,02:16,02:44
4,7,7.mp3,Girls Talk Boys,5 Seconds Of Summer,Ghostbusters (Original Motion Picture Soundtrack),29.11,02:30,02:57
...,...,...,...,...,...,...,...,...
789,993,993.mp3,Stay,Zedd,Stay,57.18,02:35,03:30
790,996,996.mp3,Trouble,offaiah,Trouble,50.17,02:05,02:53
791,997,997.mp3,A Supplementary Story : You Never Walk Alone,방탄소년단,YOU NEVER WALK ALONE,58.18,01:40,02:36
792,999,999.mp3,Outro : Wings,방탄소년단,YOU NEVER WALK ALONE,39.12,03:08,03:45


In [4]:
metadata.describe()

,musicId,duration
count,794.000000,794.000000
mean,487.847607,38.016360
std,285.248735,14.199771
min,1.000000,11.080000
25%,235.250000,28.090000
50%,485.500000,36.130000
75%,731.750000,45.140000
max,1000.000000,88.240000


In [5]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 794 entries, 0 to 793
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   musicId            794 non-null    int64  
 1   fileName           794 non-null    object 
 2   title              794 non-null    object 
 3   artist             794 non-null    object 
 4   album              787 non-null    object 
 5   duration           794 non-null    float64
 6   chorus_start_time  794 non-null    object 
 7   chorus_end_time    794 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 49.8+ KB


In [6]:
# Convert music id to string object
metadata['musicId'] = metadata['musicId'].astype(str)

# Columns to be added in metadata
lyrics = list()
lyrics_chorus = list()
avg_w_sent_song = list()
avg_w_secs_song = list()
avg_w_sent_chorus = list()
avg_w_secs_chorus = list()

# Define path for .lrc files
lyr_path = 'K:\Multi-modal-MER\PMEmo\PMEmo2019\lyrics'

# Iterate over all songs metadata
for i, mdata in metadata.iterrows():
    
    # Specific path
    filename = '\\' + mdata['musicId'] + '.lrc'
    path = lyr_path + filename
    
    # Try to read the lyric file for current song
    try:
        lyr = pd.read_csv(path, sep=']', names=['Time', 'Lyrics'])
    # Else append None to the lists
    except:
        lyrics.append(None)
        lyrics_chorus.append(None)
        avg_w_sent_chorus.append(0)
        avg_w_secs_chorus.append(0)
        avg_w_sent_song.append(0)
        avg_w_secs_song.append(0)
        continue
    
    # Basic pre-processing over the lyrics
    lyr['Time'] = lyr['Time'].str.replace('[', '')
    lyr['Time'] = lyr['Time'].str[:-3]
    lyr['Lyrics'] = lyr['Lyrics'].astype(str)
    lyr['Lyrics'] = lyr['Lyrics'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    lyr['Lyrics'] = lyr['Lyrics'].str.replace('[^\w\s*]','')
    
    num_rows = lyr.shape[0]
    last_time = 0
    
    # Concatenate the lyrics, count the rows
    all_lyrics = ''
    for j, l_data in lyr.iterrows():
        all_lyrics = all_lyrics + ' ' + l_data['Lyrics']
        if type(l_data['Time'])==str:
            last_time = l_data['Time']
        
    # Append cuurent song lyrics to list of lyrics
    lyrics.append(all_lyrics)
    
    # Check for missing values
    if not last_time or last_time==None:
        avg_w_secs_song.append(0)
        avg_w_sent_song.append(0)
    # Otherwise append
    else:
        #print(last_time, mdata['musicId'])
        try:
            duration = int(last_time.split(':')[0])*60 + int(last_time.split(':')[1].split('.')[0]) + 10
        except:
            duration = int(last_time.split(':')[0])*60 + 10
        # Average stats for the song
        avg_w_sent_song.append(len(all_lyrics.split(' '))/num_rows)
        avg_w_secs_song.append(len(all_lyrics.split(' '))/duration)
    
    # Find index of start and end time of chorus from the lyrics       
    start = mdata['chorus_start_time']
    start_index = lyr.index[lyr['Time']<=start].tolist()
    end = mdata['chorus_end_time']
    end_index = lyr.index[lyr['Time']>=end].tolist()
    
    #print(mdata['musicId'], end=' ')
    
    # Check for missing lyrics
    if len(start_index)==0 and len(end_index)==0:
        avg_w_secs_chorus.append(0)
        avg_w_sent_chorus.append(0)
    else:
        if len(end_index)==0:
            end_index.append(start_index[-1])
        elif len(start_index)==0:
            start_index.append(end_index[0])
        
        # Append chorus to corpus
        count = 0
        chorus = ''
        try:
            for l in range(start_index[-1], end_index[0]+1):
                chorus = chorus + ' ' + lyr.iloc[l]['Lyrics']
                count+=1
        except:
            print(mdata['musicId'], start_index, end_index)
            
        if count==0:
            count = 1
            
        lyrics_chorus.append(chorus)
        avg_w_sent_chorus.append(len(chorus.split(' '))/count)
        avg_w_secs_chorus.append(len(chorus.split(' '))/mdata['duration'])
        
    #print(mdata['musicId'], chorus)
        
metadata['Lyrics'] = pd.Series(lyrics)
metadata['Avg. Words/Sentence in Song'] = pd.Series(avg_w_sent_song)
metadata['Avg. Words/Second in Song'] = pd.Series(avg_w_secs_song)
metadata['Chorus Lyrics'] = pd.Series(lyrics_chorus)
metadata['Avg. Words/Sentence in Chorus'] = pd.Series(avg_w_sent_chorus)
metadata['Avg. Words/Second in Chorus'] = pd.Series(avg_w_secs_chorus)
    
metadata 
    

,musicId,fileName,title,artist,album,duration,chorus_start_time,chorus_end_time,Lyrics,Avg. Words/Sentence in Song,Avg. Words/Second in Song,Chorus Lyrics,Avg. Words/Sentence in Chorus,Avg. Words/Second in Chorus
0,1,1.mp3,Good Drank,2 Chainz,"Def Jam Presents: Direct Deposit, Vol. 2",32.10,02:35,03:05,uhh yeah alright i did it hahah yeah used to ...,6.540230,2.473913,good drank big knots good drugs i put a four ...,6.000000,2.429907
1,4,4.mp3,X Bitch (feat. Future),21 Savage,Savage Mode,28.09,03:00,03:26,None,0.000000,0.000000,None,0.000000,0.000000
2,5,5.mp3,No Heart,21 Savage,Savage Mode,84.23,00:41,02:03,hook young savage why you trappin so hard why...,8.202703,2.697778,i sit back and read like cat in the hat 21 sa...,9.666667,3.787249
3,6,6.mp3,Red Opps,21 Savage,Red Opps,29.53,02:16,02:44,yeah yeah yeah i pull up roll the window down...,8.603774,2.746988,im on that slaughter gang *****t murder gang ...,10.000000,0.338639
4,7,7.mp3,Girls Talk Boys,5 Seconds Of Summer,Ghostbusters (Original Motion Picture Soundtrack),29.11,02:30,02:57,None,0.000000,0.000000,None,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,993.mp3,Stay,Zedd,Stay,57.18,02:35,03:30,waiting for the time to pass you by hope the ...,5.740741,1.483254,all you have to do is,7.000000,0.122420
790,996,996.mp3,Trouble,offaiah,Trouble,50.17,02:05,02:53,im facing the battle nan through all of my pr...,2.666667,0.943820,nan,2.000000,0.039864
791,997,997.mp3,A Supplementary Story : You Never Walk Alone,방탄소년단,YOU NEVER WALK ALONE,58.18,01:40,02:36,None,0.000000,0.000000,None,0.000000,0.000000
792,999,999.mp3,Outro : Wings,방탄소년단,YOU NEVER WALK ALONE,39.12,03:08,03:45,None,0.000000,0.000000,None,0.000000,0.000000


In [7]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 794 entries, 0 to 793
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   musicId                        794 non-null    object 
 1   fileName                       794 non-null    object 
 2   title                          794 non-null    object 
 3   artist                         794 non-null    object 
 4   album                          787 non-null    object 
 5   duration                       794 non-null    float64
 6   chorus_start_time              794 non-null    object 
 7   chorus_end_time                794 non-null    object 
 8   Lyrics                         623 non-null    object 
 9   Avg. Words/Sentence in Song    794 non-null    float64
 10  Avg. Words/Second in Song      794 non-null    float64
 11  Chorus Lyrics                  623 non-null    object 
 12  Avg. Words/Sentence in Chorus  794 non-null    flo

In [8]:
metadata.tail()

,musicId,fileName,title,artist,album,duration,chorus_start_time,chorus_end_time,Lyrics,Avg. Words/Sentence in Song,Avg. Words/Second in Song,Chorus Lyrics,Avg. Words/Sentence in Chorus,Avg. Words/Second in Chorus
789,993,993.mp3,Stay,Zedd,Stay,57.18,02:35,03:30,waiting for the time to pass you by hope the ...,5.740741,1.483254,all you have to do is,7.0,0.122420
790,996,996.mp3,Trouble,offaiah,Trouble,50.17,02:05,02:53,im facing the battle nan through all of my pr...,2.666667,0.943820,nan,2.0,0.039864
791,997,997.mp3,A Supplementary Story : You Never Walk Alone,방탄소년단,YOU NEVER WALK ALONE,58.18,01:40,02:36,None,0.000000,0.000000,None,0.0,0.000000
792,999,999.mp3,Outro : Wings,방탄소년단,YOU NEVER WALK ALONE,39.12,03:08,03:45,None,0.000000,0.000000,None,0.0,0.000000
793,1000,1000.mp3,Spring Day,방탄소년단,YOU NEVER WALK ALONE,77.21,03:19,04:34,None,0.000000,0.000000,None,0.0,0.000000


In [9]:
metadata.head()

,musicId,fileName,title,artist,album,duration,chorus_start_time,chorus_end_time,Lyrics,Avg. Words/Sentence in Song,Avg. Words/Second in Song,Chorus Lyrics,Avg. Words/Sentence in Chorus,Avg. Words/Second in Chorus
0,1,1.mp3,Good Drank,2 Chainz,"Def Jam Presents: Direct Deposit, Vol. 2",32.10,02:35,03:05,uhh yeah alright i did it hahah yeah used to ...,6.540230,2.473913,good drank big knots good drugs i put a four ...,6.000000,2.429907
1,4,4.mp3,X Bitch (feat. Future),21 Savage,Savage Mode,28.09,03:00,03:26,None,0.000000,0.000000,None,0.000000,0.000000
2,5,5.mp3,No Heart,21 Savage,Savage Mode,84.23,00:41,02:03,hook young savage why you trappin so hard why...,8.202703,2.697778,i sit back and read like cat in the hat 21 sa...,9.666667,3.787249
3,6,6.mp3,Red Opps,21 Savage,Red Opps,29.53,02:16,02:44,yeah yeah yeah i pull up roll the window down...,8.603774,2.746988,im on that slaughter gang *****t murder gang ...,10.000000,0.338639
4,7,7.mp3,Girls Talk Boys,5 Seconds Of Summer,Ghostbusters (Original Motion Picture Soundtrack),29.11,02:30,02:57,None,0.000000,0.000000,None,0.000000,0.000000


In [10]:
metadata.to_csv('metadata_01.csv')

In [12]:
indexNames = metadata[metadata['Lyrics'].isna()].index
print(indexNames)
# Delete these row indexes from dataFrame
metadata.drop(indexNames, inplace=True)

Int64Index([  1,   4,   5,   8,  10,  11,  12,  13,  15,  23,
            ...
            777, 778, 780, 782, 783, 787, 788, 791, 792, 793],
           dtype='int64', length=171)


In [13]:
metadata

,musicId,fileName,title,artist,album,duration,chorus_start_time,chorus_end_time,Lyrics,Avg. Words/Sentence in Song,Avg. Words/Second in Song,Chorus Lyrics,Avg. Words/Sentence in Chorus,Avg. Words/Second in Chorus
0,1,1.mp3,Good Drank,2 Chainz,"Def Jam Presents: Direct Deposit, Vol. 2",32.10,02:35,03:05,uhh yeah alright i did it hahah yeah used to ...,6.540230,2.473913,good drank big knots good drugs i put a four ...,6.000000,2.429907
2,5,5.mp3,No Heart,21 Savage,Savage Mode,84.23,00:41,02:03,hook young savage why you trappin so hard why...,8.202703,2.697778,i sit back and read like cat in the hat 21 sa...,9.666667,3.787249
3,6,6.mp3,Red Opps,21 Savage,Red Opps,29.53,02:16,02:44,yeah yeah yeah i pull up roll the window down...,8.603774,2.746988,im on that slaughter gang *****t murder gang ...,10.000000,0.338639
6,9,9.mp3,Timeless,A Boogie Wit Da Hoodie,TBA,23.09,01:57,02:18,bitch im really timeless i cannot waste no ti...,8.140000,2.608974,ive been pissin codeine and i aint wit the sh...,8.300000,3.594630
7,10,10.mp3,Drowning,A Boogie Wit Da Hoodie,Drowning,32.10,02:43,03:13,im drownin im drownin im drownin im drownin w...,8.328358,2.595349,nan wrist so icy wonder why she like me bitch...,8.166667,3.052960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,983,983.mp3,Ain't My Fault,Zara Larsson,Ain't My Fault,34.12,03:12,03:44,oh my oh my oh my oh my oh my oh my oh my oh ...,7.774194,2.151786,it aint my fault,5.000000,0.146542
785,984,984.mp3,I Would Like,Zara Larsson,I Would Like,61.19,02:47,03:46,i didnt know that i could want you so deep as...,7.189655,1.837004,i would like to get to know you baby,10.000000,0.163425
786,985,985.mp3,Lush Life,Zara Larsson,Lush Life,46.13,02:37,03:21,i live my day as if it was the last live my d...,6.485714,2.247525,i live my day as if it was the last,11.000000,0.238457
789,993,993.mp3,Stay,Zedd,Stay,57.18,02:35,03:30,waiting for the time to pass you by hope the ...,5.740741,1.483254,all you have to do is,7.000000,0.122420


In [14]:
metadata.to_csv('metadata_02.csv')

In [15]:
metadata['Lyrics']

0       uhh yeah alright i did it hahah yeah used to ...
2       hook young savage why you trappin so hard why...
3       yeah yeah yeah i pull up roll the window down...
6       bitch im really timeless i cannot waste no ti...
7       im drownin im drownin im drownin im drownin w...
                             ...                        
784     oh my oh my oh my oh my oh my oh my oh my oh ...
785     i didnt know that i could want you so deep as...
786     i live my day as if it was the last live my d...
789     waiting for the time to pass you by hope the ...
790     im facing the battle nan through all of my pr...
Name: Lyrics, Length: 623, dtype: object

## Rough

In [1]:
lrc_file = open(r'K:\Multi-modal-MER\PMEmo\PMEmo2019\lyrics\1.lrc')

In [2]:
lrc_string = ''.join(lrc_file.readlines())
lrc_file.close()

In [3]:
lrc_string

"[by:Trap_Girl]\n[00:03.00]Uhh, (yeah) alright\n[00:12.28]I did it, hahah, yeah\n[00:14.62]Used to treat my mattress like the ATM, yeah\n[00:18.34]Bond number 9 that's my favorite scent, yeah\n[00:21.93]Can't forget the kush, I'm talking OG, yeah\n[00:25.64]Rest in peace to pop, he was an OG, oh yeah\n[00:29.40]285 I had that pack on me, uh\n[00:33.07]I cannot forget I had that strap on me, yeah\n[00:36.67]Rest in peace to my nigga Doe (Doe TRU)\n[00:40.49]All we ever want to do is ball\n[00:43.23]That was the easy part\n[00:45.34]We playing that Weezy hard\n[00:47.03]We sit in the kitchen late\n[00:48.84]We tryna to make an escape\n[00:50.80]Trying to make me a mil\n[00:52.66]So I'ma keep me a plate\n[00:54.47]I told 'em shawty can leave\n[00:56.46]So I'mma keep me a rake\n[00:58.24]So I'ma keep me a Wraith\n[01:00.05]My jewelry look like a lake\n[01:02.01]Today I'm in the Maybach\n[01:03.72]And that car came with some drapes\n[01:05.66]You know I look like a safe\n[01:07.40]I put you

In [4]:
ind = lrc_string.find(']')
lrc_string = lrc_string[ind+2:]
#lrc_string = lrc_string.replace('', ' ')
lrc_string = lrc_string.replace(']', ']$')
lrc_string

"[00:03.00]$Uhh, (yeah) alright\n[00:12.28]$I did it, hahah, yeah\n[00:14.62]$Used to treat my mattress like the ATM, yeah\n[00:18.34]$Bond number 9 that's my favorite scent, yeah\n[00:21.93]$Can't forget the kush, I'm talking OG, yeah\n[00:25.64]$Rest in peace to pop, he was an OG, oh yeah\n[00:29.40]$285 I had that pack on me, uh\n[00:33.07]$I cannot forget I had that strap on me, yeah\n[00:36.67]$Rest in peace to my nigga Doe (Doe TRU)\n[00:40.49]$All we ever want to do is ball\n[00:43.23]$That was the easy part\n[00:45.34]$We playing that Weezy hard\n[00:47.03]$We sit in the kitchen late\n[00:48.84]$We tryna to make an escape\n[00:50.80]$Trying to make me a mil\n[00:52.66]$So I'ma keep me a plate\n[00:54.47]$I told 'em shawty can leave\n[00:56.46]$So I'mma keep me a rake\n[00:58.24]$So I'ma keep me a Wraith\n[01:00.05]$My jewelry look like a lake\n[01:02.01]$Today I'm in the Maybach\n[01:03.72]$And that car came with some drapes\n[01:05.66]$You know I look like a safe\n[01:07.40]$I

In [5]:
StringData = StringIO(lrc_string)
lyr = pd.read_csv(StringData, sep='$', names=['Time', 'Lyrics'])
lyr

NameError: name 'StringIO' is not defined

In [ ]:
lyr['Time'] = lyr['Time'].str.replace('[', '')
lyr['Time'] = lyr['Time'].str.replace(']', '')
lyr

In [ ]:
lyr['Lyrics'] = lyr['Lyrics'].apply(lambda x: " ".join(x.lower() for x in x.split()))
lyr['Lyrics'] = lyr['Lyrics'].str.replace('[^\w\s]','')
lyr['Lyrics']

In [ ]:
lyr['Number of Words'] = 0
lyr.head()

In [ ]:
lyr['Number of Words'] = lyr['Lyrics'].str.split(' ').str.len()
lyr

In [ ]:
lyr['Number of Words per second'] = 0

In [ ]:
for i, row in lyr.iterrows():
    starttime = row['Time'].split(':')
    start = 60*starttime[0] + int(starttime[1].split('.')[0])
    if i<len(lyr)-1:
        endtime = lyr.iloc[i+1]['Time'].split(':')
        end = 60*endtime[0] + int(endtime[1].split('.')[0])
    else:
        endtime = metadata
    print(starttime, endtime)

In [9]:
file = pd.read_csv(r'K:\Multi-modal-MER\PMEmo\PMEmo2019\lyrics\108.lrc', delimiter=']')
file

,[,Unnamed: 1
0,[00:00.920,"Baby, this is what you came for"
1,[00:04.130,Lightning strikes every time she moves
2,[00:11.620,And everybody’s watching her
3,[00:14.170,"But she’s looking at you, oh, oh"
4,[00:17.970,"You, oh, oh, you, oh, oh"
5,[00:29.020,"Oh, oh"
6,[00:32.760,"Baby,this is what you came for"
7,[00:35.680,Lightening strikes every time she moves
8,[00:42.740,And everybody’s watching her
9,[00:45.510,"But she’s looking at you, oh, oh"


In [ ]:
    try:
        lrc_file = open(path)
    except:
        continue
        
    #print(mdata, i)
    
    lrc_string = ''.join(lrc_file.readlines())
    lrc_file.close()
    
    ind = lrc_string.find(']')
    lrc_string = lrc_string[ind+2:]
    lrc_string = lrc_string.replace(']', ']$')
    
    StringData = StringIO(lrc_string)
    lyr = pd.read_csv(StringData, sep='$', names=['Time', 'Lyrics'])